In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime as dt
from pandas_datareader import DataReader as DR
import seaborn as sb
import numdifftools as nd
from wquantiles import quantile
import statsmodels.api as sm

from scipy.stats import norm,t,truncnorm
from scipy.stats import multivariate_normal as mvnorm
from scipy.stats import multivariate_t as mvt
from scipy.spatial import Delaunay as TRI
from scipy.interpolate import LinearNDInterpolator as ITP
from scipy.optimize import minimize,root
from scipy.optimize import NonlinearConstraint as NonlinCons
from scipy.stats import gaussian_kde as sciKDE

from sklearn.linear_model import LinearRegression as Linear
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KernelDensity as sklKDE

import warnings
warnings.filterwarnings("ignore")

# Define the experiment class

In [ ]:
class MLE:
    def __init__(self,dim,sigma):
        self.dim=dim
        self.T=lambda x: mvnorm.pdf(x=x,mean=np.zeros(dim))
        self.iP=lambda x: mvnorm.pdf(x=x,mean=np.zeros(dim),cov=sigma**2)
        self.iS=lambda size: mvnorm.rvs(size=size,mean=np.zeros(dim),cov=sigma**2)
        
    def __estimate(self,W,name,asym=True):
        Z=np.mean(W)
        err=np.abs(Z-1)
        if asym:
            aVar=np.var(W)
            aErr=np.sqrt(aVar/W.size)
            ESS=1/np.sum((W/np.sum(W))**2)
            print('{} est: {:.4f}; err: {:.4f}; a-var: {:.4f}; a-err: {:.4f}; ESS: {:.0f}/{}'\
                  .format(name,Z,err,aVar,aErr,ESS,W.size))
        else:
            print('{} est: {:.4f}; err: {:.4f}'.format(name,Z,err))
        
    def estimate_IS(self,size):
        S=self.iS(size)
        W=self.T(S)/self.iP(S)
        self.__estimate(W,'IS')
    
    def draw_TP(self,P,x,name,dim=0):
        X=np.zeros([x.size,self.dim])
        X[:,dim]=x
        ratio=np.reshape(self.T(np.zeros(self.dim))/P(np.zeros([1,self.dim])),1)[0]
        print('------------ pdf ratio at origin: {:.2f} ------------'.format(ratio))
        
        fig,ax=plt.subplots(figsize=(7,4))
        ax.plot(x,self.T(X))
        ax.plot(x,P(X))
        if name=='nonparametric':
            one=np.zeros(self.dim)
            one[dim]=1
            rW=np.array([self.h(one*loc,loc) for loc in self.rS])
            rW=rW/rW.max()*P(np.zeros([1,self.dim]))[0]
            rWmeans=np.ones_like(rW)*rW.mean()

            ax.plot(x,self.mP(X))
            ax.hist(self.rS[:,dim],bins=2*rW.size,weights=rWmeans)
            ax.hist(self.rS[:,dim],bins=2*rW.size,weights=rW)
            ax.legend(['target','nonparametric proposal','mixture proposal','centers','centers with weight'])
        elif name=='regression':
            G=self.G(X)
            rPO=self.regO.coef_.dot(G)+self.regO.intercept_*P(X)
            rPL=self.regL.coef_.dot(G)+self.regL.intercept_*P(X)
            mid=int(x.size/2)
            print('regression ratios: ordinary {:.4f}, lasso {:.4f}'\
                  .format(self.T(X[mid])/rPO[mid],self.T(X[mid])/rPL[mid]))
            
            ax.plot(x,rPO)
            ax.plot(x,rPL)
            ax.legend(['target','mixture proposal','ordinary regression','lasso regression'])
        else:
            ax.legend(['target','{} proposal'.format(name)])
            
        ax.set_title('{}-D target and {} proposal (cross-sectional view)'.format(self.dim,name))
        plt.show()
        
    def resample(self,size,ratio):
        S=self.iS(ratio*size)
        p=self.T(S)/self.iP(S)
        index=np.arange(S.shape[0])
        choice=np.random.choice(index,size,p=p/np.sum(p),replace=True)
        
        self.rS=S[choice]
        self.rSset=S[list(set(choice))]
        print('resampling rate: {}/{}'.format(self.rSset.shape[0],size))
        
    def estimate_NIS(self,size,rate,bdwth='scott'):
        if(type(bdwth)==str):
            method=bdwth
            tmp=sciKDE(self.rS.T,bw_method=method)
            bdwth=np.mean(np.sqrt(np.diag(tmp.covariance_factor()*np.cov(self.rS.T))))
            print('bdwth: {:.4f} (based on {})'.format(bdwth,method))
        
        self.bdwth=bdwth
        self.kde=sklKDE(kernel='gaussian',bandwidth=bdwth).fit(self.rS)
        self.h=lambda x,loc: mvnorm.pdf(x=x,mean=loc,cov=self.bdwth**2)
        self.G=lambda x: np.array([self.h(x,loc) for loc in self.rSset])-self.iP(x)
        
        self.nP=lambda x: np.exp(self.kde.score_samples(x))
        self.nS=lambda size: self.kde.sample(size)
        S=self.nS(size)
        W=self.T(S)/self.nP(S)
        self.__estimate(W,'NIS')
        
        self.mP=lambda x: (1-rate)*self.iP(x)+rate*self.nP(x)
        self.mS=lambda size: np.vstack([self.iS(size-round(rate*size)),self.nS(round(rate*size))])
        self.S=self.mS(size)
        W=self.T(self.S)/self.mP(self.S)
        self.__estimate(W,'MIS')
        
    def estimate_RIS(self,alpha):
        X=(self.G(self.S)/self.mP(self.S)).T
        y=self.T(self.S)/self.mP(self.S)
        self.regO=Linear().fit(X,y)
        self.regL=Lasso(alpha).fit(X,y)
        print('Ordinary R2: {:.4f}; Lasso R2: {:.4f}'.format(self.regO.score(X,y),self.regL.score(X,y)))
        
        W=y-X.dot(self.regO.coef_)
        self.__estimate(W,'RIS(Ord)')
        W=y-X.dot(self.regL.coef_)
        self.__estimate(W,'RIS(Las)')
    
    def estimate_MLE(self,opt=False,init=0):
        mP=self.mP(self.S)
        G=self.G(self.S)
        target=lambda zeta: -np.mean(np.log(mP+zeta.dot(G)))
        gradient=lambda zeta: -np.mean(G/(mP+zeta.dot(G)),axis=1)
        hessian=lambda zeta: (G/(mP+zeta.dot(G))**2).dot(G.T)/G.shape[1]
        zeta0=np.zeros(G.shape[0])
        grad0=gradient(zeta0)
        print('Reference:')
        print('origin: value: {:.4f}; grad: (min {:.4f}, mean {:.4f}, max {:.4f}, std {:.4f})'\
              .format(target(zeta0),grad0.min(),grad0.mean(),grad0.max(),grad0.std()))
        
        print()
        print('Theoretical results:')
        X=(G/mP).T
        XX=X-X.mean(axis=0)
        zeta1=np.linalg.solve(XX.T.dot(XX),X.sum(axis=0))
        print('MLE(The) zeta: (min {:.4f}, mean {:.4f}, max {:.4f}, std {:.4f}, norm {:.4f})'\
              .format(zeta1.min(),zeta1.mean(),zeta1.max(),zeta1.std(),np.sqrt(np.sum(zeta1**2))))
        grad1=gradient(zeta1)
        print('theory: value: {:.4f}; grad: (min {:.4f}, mean {:.4f}, max {:.4f}, std {:.4f})'\
              .format(target(zeta1),grad1.min(),grad1.mean(),grad1.max(),grad1.std()))
        W=(self.T(self.S)/mP)*(1-XX.dot(zeta1))
        self.__estimate(W,'RIS(The)',asym=False)
        W=self.T(self.S)/(mP+zeta1.dot(G))
        self.__estimate(W,'MLE(The)',asym=False)
        
        if opt:
            zeta=zeta1 if init==1 else zeta0
            begin=dt.now()
            res=root(lambda zeta: (gradient(zeta),hessian(zeta)),zeta,method='lm',jac=True)
            end=dt.now()
            print()
            print('Optimization results (spent {} seconds):'.format((end-begin).seconds))
            if res['success']:
                zeta=res['x']
                print('MLE(Opt) zeta: (min {:.4f}, mean {:.4f}, max {:.4f}, std {:.4f}, norm {:.4f})'\
                      .format(zeta.min(),zeta.mean(),zeta.max(),zeta.std(),np.sqrt(np.sum(zeta**2))))
                print('Dist(zeta(Opt),zeta(The))={:.4f}'.format(np.sqrt(np.sum((zeta-zeta1)**2))))
                grad=gradient(zeta)
                print('optimal: value: {:.4f}; grad: (min {:.4f}, mean {:.4f}, max {:.4f}, std {:.4f})'\
                      .format(target(zeta),grad.min(),grad.mean(),grad.max(),grad.std()))
                W=self.T(self.S)/(mP+zeta.dot(G))
                self.__estimate(W,'MLE(Opt)',asym=False)
            else:
                print('MLE fail')

**Limitations:**
1. didn't consider self-normalized importance sampling
2. only for symmetric normal target and symmetric normal initial proposal with only one mode
3. only for normal KDE without weights and adaptive bandwidth
4. only consider regression for proposal components based on mixture proposal

# Run the experiments

## Initial proposal and the curse of dimensionality

In [ ]:
mle=MLE(dim=2,sigma=2)
size=1000000
mle.estimate_IS(size)
x=np.linspace(-4,4,101)
mle.draw_TP(mle.iP,x,'initial')

print('=======================================================')
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('=======================================================')

mle=MLE(dim=4,sigma=2)
mle.estimate_IS(size)
mle.draw_TP(mle.iP,x,'initial')

print('=======================================================')
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('=======================================================')

mle=MLE(dim=8,sigma=2)
mle.estimate_IS(size)
mle.draw_TP(mle.iP,x,'initial')

**Summary:**
* The cumulative phenomenon of scale difference of all dimension. 

**Future:**
* What about the comparison between multi-normal and multi-t? 

## KDE with different initial proposal and bandwidth in 6-dimension

In [ ]:
np.random.seed(1234)
mle=MLE(dim=6,sigma=1)
size=50000
mle.estimate_IS(size)
x=np.linspace(-4,4,101)
mle.draw_TP(mle.iP,x,'initial')
print('=======================================================')
mle.resample(1000,20)
mle.estimate_NIS(size,0.9)
mle.draw_TP(mle.nP,x,'nonparametric')

print('=======================================================')
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('=======================================================')

np.random.seed(1234)
mle=MLE(dim=6,sigma=2)
mle.estimate_IS(size)
mle.draw_TP(mle.iP,x,'initial')
print('=======================================================')
mle.resample(1000,20)
np.random.seed(123456)
mle.estimate_NIS(size,0.9)
mle.draw_TP(mle.nP,x,'nonparametric')

In [ ]:
np.random.seed(123456)
mle.estimate_NIS(size,0.9,0.6)
mle.draw_TP(mle.nP,x,'nonparametric')

print('=======================================================')
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('=======================================================')

np.random.seed(123456)
mle.estimate_NIS(size,0.9,0.5)
mle.draw_TP(mle.nP,x,'nonparametric')

print('=======================================================')
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('=======================================================')

np.random.seed(123456)
mle.estimate_NIS(size,0.9,0.4)
mle.draw_TP(mle.nP,x,'nonparametric')

print('=======================================================')
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('=======================================================')

np.random.seed(123456)
mle.estimate_NIS(size,0.8,0.4)
mle.draw_TP(mle.nP,x,'nonparametric')

**Summary:**
* Resampling ratio 20:1 can give us good 'samples' from the target. (We can increase it for better resample rate in higher dimension. )
* Only conservative bandwidth can work well in higher dimension, because little bandwidth can raise serious tail problem. 

**Future**
* What about KDE with weights? 
* What about the adaptive bandwidth? 

## Regression performance visualization

In [ ]:
np.random.seed(1234)
mle=MLE(dim=8,sigma=2)
size=100000
mle.estimate_IS(size)
x=np.linspace(-4,4,101)
mle.draw_TP(mle.iP,x,'initial')
print('=======================================================')
mle.resample(1000,20)
mle.estimate_NIS(size,0.9)
mle.draw_TP(mle.nP,x,'nonparametric')
print('=======================================================')
mle.estimate_RIS(0.1)
mle.draw_TP(mle.mP,x,'regression')

In [ ]:
np.random.seed(1234)
mle=MLE(dim=10,sigma=2)
mle.estimate_IS(size)
mle.draw_TP(mle.iP,x,'initial')
print('=======================================================')
np.random.seed(123456)
mle.resample(1000,20)
mle.estimate_NIS(size,0.9)
mle.draw_TP(mle.nP,x,'nonparametric')
print('=======================================================')
mle.estimate_RIS(0.1)
mle.draw_TP(mle.mP,x,'regression')

In [ ]:
np.random.seed(123456)
mle.resample(1000,100)
mle.estimate_NIS(size,0.9)
mle.draw_TP(mle.nP,x,'nonparametric')
print('=======================================================')
mle.estimate_RIS(0.1)
mle.draw_TP(mle.mP,x,'regression')

**Summary:**
* Regression improve the estimation performance by mimic the shape of the target. 
* In higher dimension, improve the quality of kernels by increasing resampling ratio can slightly improve KDE but more dramatically improve regression. 

**Future**
* The problem of singularity? (What about multi-t initial? )
* What about regression for kernels not used in KDE? 
* What about regression with KDE with weights? 
* What about regression for kernels with adaptive bandwidth? (Will that improve the kernels' quality? )

## MLE method investigation

In [ ]:
np.random.seed(1234)
mle=MLE(dim=10,sigma=2)
size=100000
mle.estimate_IS(size)
x=np.linspace(-4,4,101)
mle.draw_TP(mle.iP,x,'initial')
print('=======================================================')
mle.resample(1000,100)
mle.estimate_NIS(size,0.9)
mle.draw_TP(mle.nP,x,'nonparametric')
print('=======================================================')
mle.estimate_RIS(0.1)
mle.draw_TP(mle.mP,x,'regression')
print('=======================================================')
mle.estimate_MLE(True)

In [ ]:
np.random.seed(1234)
mle=MLE(dim=10,sigma=1)
mle.estimate_IS(size)
mle.draw_TP(mle.iP,x,'initial')
print('=======================================================')
mle.resample(1000,100)
mle.estimate_NIS(size,0.9)
mle.draw_TP(mle.nP,x,'nonparametric')
print('=======================================================')
mle.estimate_RIS(0.1)
mle.draw_TP(mle.mP,x,'regression')
print('=======================================================')
mle.estimate_MLE(True)

In [ ]:
np.random.seed(1234)
mle=MLE(dim=5,sigma=2)
size=20000
mle.estimate_IS(size)
x=np.linspace(-4,4,101)
mle.draw_TP(mle.iP,x,'initial')
print('=======================================================')
mle.resample(1000,100)
mle.estimate_NIS(size,0.9)
mle.draw_TP(mle.nP,x,'nonparametric')
print('=======================================================')
mle.estimate_RIS(0.1)
mle.draw_TP(mle.mP,x,'regression')
print('=======================================================')
mle.estimate_MLE(True,0)

print('=======================================================')
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('=======================================================')

mle.estimate_MLE(True,1)

**Summary:**
* Theoretical zeta is very close to optimal zeta, so we can use it as the initial guess for optimization. 
* Root finding for the gradient is far more efficient than maximization. 
* Theoretical RIS give exact same result as regression. 
* Theoretical MLE can achieve similar results as MLE based on optimization, so we can use it as an alternative method. 
* Theoretical MLE can't give exact result when target can be expressed by proposals, because its gradients are not small enough. 
* Sometimes (low-dimension, many kernels and small sample size? ) the theoretical zeta can be out of the feasible region, which indicates that the optimal zeta is very close to the boundary. 
* A bad theoretical zeta still is a good initial guess for optimization, which may result in infeasible but usable optimization result. 

**Future**
* Try more detailed experiments for MLE. 